In [ ]:
"""

Compare the melt resulting from old and new tuning for last 10 years of
OPM026,OPM031,ctrl94,isf94

"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import cmocean
import glob

In [ ]:
%matplotlib qt5

In [ ]:
home_path = '/bettik/burgardc/'

In [ ]:
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_OPM016/'
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    
home_path = '/bettik/burgardc/'
plot_path='/bettik/burgardc/PLOTS/summer_paper_plots/'

In [ ]:
nemo_run = 'isf94'


## Melt outputpath
outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CHECK_TUNING/nemo_5km_'+nemo_run+'/'

ds_melt_param_Gt_new = []
ds_melt_param_box1_new = []
ds_melt_param_Gt_old = []
ds_melt_param_box1_old = []

melt_newtuning_classic = list(sorted(glob.glob(outputpath_melt+'eval_metrics_1D_*_newtuning_oneFRIS.nc')))
melt_oldtuning_classic = list(sorted(glob.glob(outputpath_melt+'eval_metrics_1D_*_oldtuning_oneFRIS.nc')))

param_list = []

for mfilename in melt_newtuning_classic:
    #paramname = mfilename[97:157].split('.')[0].split('_')[:-1]
    if nemo_run == 'isf94':
        paramname = mfilename[105:157].split('.')[0].split('_')[:-2]
    elif nemo_run == 'isfru94':
        paramname = mfilename[107:157].split('.')[0].split('_')[:-2]
    else:
        paramname = mfilename[106:157].split('.')[0].split('_')[:-2]
    #paramname = mfilename[105:157].split('.')[0].split('_')[:-1]
    mparam = '_'.join(paramname)
    param_list.append(mparam)

melt_new_classic  = xr.open_mfdataset(melt_newtuning_classic, concat_dim='new_param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
melt_new_classic = melt_new_classic.rename({'new_param': 'param'})
melt_new_classic = melt_new_classic.assign_coords(param=param_list) 
ds_melt_param_Gt_new.append(melt_new_classic['melt_1D_Gt_per_y'])
ds_melt_param_box1_new.append(melt_new_classic['melt_1D_mean_myr_box1'])

melt_old_classic  = xr.open_mfdataset(melt_oldtuning_classic, concat_dim='new_param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
melt_old_classic = melt_old_classic.rename({'new_param': 'param'})
melt_old_classic = melt_old_classic.assign_coords(param=param_list)
ds_melt_param_Gt_old.append(melt_old_classic['melt_1D_Gt_per_y'])
ds_melt_param_box1_old.append(melt_old_classic['melt_1D_mean_myr_box1'])

melt_new_NN_list = []
melt_old_NN_list = []
for nn_type in ['xsmall96','small','large']:
    melt_newtuning_nntype = xr.open_mfdataset(outputpath_melt+'evalmetrics_1D_'+nn_type+'_*_newtuning.nc')
    melt_newtuning_nntype = melt_newtuning_nntype.sel(time=melt_newtuning_nntype.time[-10::]).load()
    melt_oldtuning_nntype = xr.open_mfdataset(outputpath_melt+'evalmetrics_1D_'+nn_type+'_*_oldtuning.nc').sel(time=melt_newtuning_nntype.time).load()
    melt_new_NN_list.append(melt_newtuning_nntype.assign_coords({'param': nn_type}))
    melt_old_NN_list.append(melt_oldtuning_nntype.assign_coords({'param': nn_type}))
melt_new_NN = xr.concat(melt_new_NN_list, dim='param')
ds_melt_param_Gt_new.append(melt_new_NN['predicted_melt'].sel(metrics='Gt').squeeze().drop('metrics'))
ds_melt_param_box1_new.append(melt_new_NN['predicted_melt'].sel(metrics='Gt').squeeze().drop('metrics'))

melt_old_NN = xr.concat(melt_old_NN_list, dim='param')
ds_melt_param_Gt_old.append(melt_old_NN['predicted_melt'].sel(metrics='Gt').squeeze().drop('metrics'))
ds_melt_param_box1_old.append(melt_old_NN['predicted_melt'].sel(metrics='Gt').squeeze().drop('metrics'))

ds_melt_param_Gt_new.append(melt_new_NN['reference_melt'].sel(metrics='Gt',param='small').squeeze().drop(['metrics','param']).assign_coords({'param':'reference'}))
ds_melt_param_box1_new.append(melt_new_NN['reference_melt'].sel(metrics='Gt',param='small').squeeze().drop(['metrics','param']).assign_coords({'param':'reference'}))

ds_melt_param_Gt_old_nrun = xr.concat(ds_melt_param_Gt_old, dim='param').sel(profile_domain=50)
ds_melt_param_box1_old_nrun = xr.concat(ds_melt_param_box1_old, dim='param').sel(profile_domain=50)
ds_melt_param_Gt_new_nrun = xr.concat(ds_melt_param_Gt_new, dim='param')
ds_melt_param_box1_new_nrun = xr.concat(ds_melt_param_box1_new, dim='param')


In [ ]:
ds_melt_param_Gt_old_nrun.sel(param='reference', Nisf=kisf)

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','grey','yellowgreen']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

sorted_isf_all = [11,69,43,12,70,44,13,58,71,45,30,31,61,73,47,48,33,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,42,55]

for kisf in tqdm(sorted_isf_all):
    
    ax[i] = f.add_subplot(6,6,i+1)
    ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_new_nrun.sel(param='reference', Nisf=kisf), color='k')

    
    for n,mparam in enumerate(ds_melt_param_Gt_new_nrun.param):
        
        ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_new_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color='red',alpha=0.5,
                                                                  label=mparam.values)
        ax[i].plot(ds_melt_param_Gt_old_nrun.time, ds_melt_param_Gt_old_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color='orange',
                                                               alpha=0.5)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_compare_old_new_tuning_'+nemo_run+'.pdf')

In [ ]:
ds_melt_param_Gt_new_nrun.param

In [ ]:
simple_param_list = ['linear_local','quadratic_local','quadratic_local_locslope']
complex_param_list = ['lazero19','boxes_4_pismyes_picopno']
nn_param_list = ['xsmall96','small','large']
all_param_list = simple_param_list + complex_param_list + nn_param_list

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['maroon','orange','magenta']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

sorted_isf_all = [11,69,43,12,70,44,13,58,71,45,30,31,61,73,47,48,33,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,42,55]

for kisf in tqdm(sorted_isf_all):
    
    ax[i] = f.add_subplot(6,6,i+1)
    ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_old_nrun.sel(param='reference', Nisf=kisf), color='k')

    
    for n,mparam in enumerate(ds_melt_param_Gt_new_nrun.param.sel(param=simple_param_list)):
        
        ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_new_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],alpha=0.7,
                                                                  label=mparam.values)
        ax[i].plot(ds_melt_param_Gt_old_nrun.time, ds_melt_param_Gt_old_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],linestyle='--',
                                                               alpha=0.7)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_compare_old_new_tuning_simpleparams_'+nemo_run+'.pdf')

In [ ]:
ds_melt_param_Gt_new_nrun.param

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['deepskyblue','royalblue']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

sorted_isf_all = [11,69,43,12,70,44,13,58,71,45,30,31,61,73,47,48,33,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,42,55]

for kisf in tqdm(sorted_isf_all):
    
    ax[i] = f.add_subplot(6,6,i+1)
    ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_new_nrun.sel(param='reference', Nisf=kisf), color='k')

    
    for n,mparam in enumerate(ds_melt_param_Gt_new_nrun.param.sel(param=complex_param_list)):
        
        ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_new_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],alpha=0.7,
                                                                  label=mparam.values)
        ax[i].plot(ds_melt_param_Gt_old_nrun.time, ds_melt_param_Gt_old_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],linestyle='--',
                                                               alpha=0.7)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_compare_old_new_tuning_complexparams_'+nemo_run+'.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['limegreen','olive','yellowgreen']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

sorted_isf_all = [11,69,43,12,70,44,13,58,71,45,30,31,61,73,47,48,33,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,42,55]

for kisf in tqdm(sorted_isf_all):
    
    ax[i] = f.add_subplot(6,6,i+1)
    ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_old_nrun.sel(param='reference', Nisf=kisf), color='k')

    
    for n,mparam in enumerate(ds_melt_param_Gt_new_nrun.param.sel(param=nn_param_list)):
        
        ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_new_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],alpha=0.7,
                                                                  label=mparam.values)
        ax[i].plot(ds_melt_param_Gt_old_nrun.time, ds_melt_param_Gt_old_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],linestyle='--',
                                                               alpha=0.7)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_compare_old_new_tuning_nnparams_'+nemo_run+'.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['maroon','orange','magenta','deepskyblue','royalblue','limegreen','olive','green']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

sorted_isf_all = [11,69,43,12,70,44,13,58,71,45,30,31,61,73,47,48,33,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,42,55]

for kisf in tqdm(sorted_isf_all):
    
    ax[i] = f.add_subplot(6,6,i+1)
    ax[i].plot(ds_melt_param_Gt_old_nrun.time, ds_melt_param_Gt_old_nrun.sel(param='reference', Nisf=kisf), color='k')

    
    for n,mparam in enumerate(ds_melt_param_Gt_new_nrun.param.sel(param=all_param_list)):
        
        ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_new_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],alpha=0.7,
                                                                  label=mparam.values)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_compare_onlynew_tuning_'+nemo_run+'.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['maroon','orange','magenta','deepskyblue','royalblue','limegreen','olive','green']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

sorted_isf_all = [11,69,43,12,70,44,13,58,71,45,30,31,61,73,47,48,33,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,42,55]

for kisf in tqdm(sorted_isf_all):
    
    ax[i] = f.add_subplot(6,6,i+1)
    ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_old_nrun.sel(param='reference', Nisf=kisf), color='k')

    
    for n,mparam in enumerate(ds_melt_param_Gt_new_nrun.param.sel(param=all_param_list)):
        
        ax[i].plot(ds_melt_param_Gt_old_nrun.time, ds_melt_param_Gt_old_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],alpha=0.7,
                                                                  label=mparam.values,linestyle='--')


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_compare_onlyold_tuning_'+nemo_run+'.pdf')